In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# Jenny's codes start here
vaccine_county = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)

In [3]:
vaccine_county_rename = vaccine_county.rename(columns={"Recip_County": "County", "Series_Complete_Pop_Pct": "Vaccination_Complete_Pct","Recip_State":"State"})
vaccine_county_rename.shape

(2954, 32)

In [4]:
svi_df = pd.read_csv("Resource/04_siv_county.csv",low_memory=False)
svi_df = svi_df.dropna(subset=['Social Vulnerability Index (SVI)'])
svi_df = svi_df.loc[svi_df['Social Vulnerability Index (SVI)']!=0]
svi_df_rename = svi_df.rename(columns={"FIPS Code": "FIPS", "County Name": "County"})
svi_df_rename.shape

(3125, 10)

In [5]:
svi_df_rename['County'] = svi_df_rename['County'].str.split(',', expand=True)[0]

In [6]:
vaccine_svi_df = pd.merge(vaccine_county_rename, svi_df_rename, on='FIPS')
vaccine_svi_df = vaccine_svi_df[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','Metro_status','Social Vulnerability Index (SVI)','SVI Category']] 
vaccine_svi_df = vaccine_svi_df.rename(columns={"County_x": "County","State_x": "State"})

In [7]:
us_county = pd.read_csv("Resource/05_us_county_coord.csv",low_memory=False)
us_county_rename = us_county.rename(columns={"fips": "FIPS"})
us_county_rename = us_county_rename[['FIPS','lat','long']]

In [8]:
vaccine_svi_df = pd.merge(vaccine_svi_df, us_county_rename, on='FIPS')
vaccine_svi_df.shape

(2858, 10)

In [9]:
# Jenny's codes end here
vaccine_svi_df.to_csv("Clean_data/vaccine_svi_df.csv", encoding="utf-8", index=False)